In [1]:
#!/usr/bin/env python3
"""
NMC界面構造用MD計算システム統合版
統合NMC-Al界面構造生成システムで生成された構造に対応

主な修正点:
1. 入力ファイル形式をCIFに変更
2. NMC組成バリアント名に対応
3. ファイルパス構造を統合システムに合わせて変更
4. 界面構造に適した固定層設定
"""

from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pandas as pd
from pathlib import Path
from ase import units
from ase.build import add_vacuum
import re

# =============================================
# 統合システム対応のファイル・ディレクトリパス設定
# =============================================

# ベースディレクトリ（統合システムと同期）
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# 統合システムで生成された界面構造の入力ディレクトリ
# INPUT_DIR = os.path.join(BASE_DIR, "structure", "output", "cif_files")
INPUT_DIR = os.path.join(BASE_DIR, "structure","AlF3_NMC")
# Kaori/MD/LiB_2/structure/output/cif_files/Interface_AlF3_on_NMC_pristine_104.cif
# MD計算結果の出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "structure/MD_AlF3")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile01")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# =============================================
# 統合システム対応のファイルパス関数
# =============================================

def get_interface_files():
    """統合システムで生成された界面構造ファイルを取得"""
    interface_files = []
    if os.path.exists(INPUT_DIR):
        cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.cif')]
        for cif_file in cif_files:
            # ファイル名から材料情報を抽出
            # 例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
            if "Interface_" in cif_file and "_on_" in cif_file:
                interface_files.append(cif_file)
    # interface_files = []
    # if os.path.exists(INPUT_DIR):
    #     cif_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.xyzf')]
    #     for cif_file in cif_files:
    #         # ファイル名から材料情報を抽出
    #         # 例: "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
    #         if "Interface_" in cif_file and "_on_" in cif_file:
    #             interface_files.append(cif_file)
    print(f"発見された界面構造ファイル数: {len(interface_files)}")
    return interface_files

def parse_interface_filename(filename):
    """
    界面構造ファイル名から材料情報を解析（AlF3, Al2O3などにも対応）
    ファイル名例: 
    - "Interface_Al_111_on_NMC111_Ni_increase_001_trimmed.cif"
    - "Interface_AlF3_on_NMC_pristine_104.cif"
    """
    if not filename.startswith("Interface_"):
        # 始まっていない場合は、メッセージを出力して処理を中断
        print(f"⏩ スキップ: ファイル名が 'Interface_' で始まらないため処理対象外です: {filename}")
        return None
    base_name_for_error = Path(filename).stem
    
    try:
        cleaned_name = re.sub(r'\.cif$', '', filename)
        cleaned_name = re.sub(r'^Interface_', '', cleaned_name)

        parts = cleaned_name.split('_on_')
        al_part_str = parts[0]
        nmc_part_str = parts[1]
        
        # --- Al側の解析（修正箇所） ---
        al_parts = al_part_str.split('_')
        
        # al_partsの要素数で分岐
        if len(al_parts) > 1:
            # 要素が複数なら、最後がミラー指数、それ以外が物質名
            miller_al = al_parts[-1]
            al_component = "_".join(al_parts[:-1])
        else:
            # 要素が1つなら、それが物質名で、ミラー指数はなし
            al_component = al_parts[0]
            miller_al = "bulk"  # ミラー指数がない場合は 'bulk' とする

        # --- NMC側の解析 ---
        nmc_parts = nmc_part_str.split('_')
        nmc_component = "NMC" 
        nmc_variant = "pristine"
        
        if "NMC" in nmc_parts[0]:
            nmc_component = nmc_parts.pop(0)

        miller_nmc = nmc_parts.pop(-1)
        if nmc_parts and nmc_parts[-1] == 'trimmed':
            nmc_parts.pop(-1)
            
        if nmc_parts:
            nmc_variant = "_".join(nmc_parts)

    except (ValueError, IndexError) as e:
        print(f"⚠️ ファイル名解析警告 ({filename}): {e}")
        return {
            'al_component': "unknown", 'nmc_component': "unknown",
            'nmc_variant': "unknown", 'miller_al': "unknown",
            'miller_nmc': "unknown", 'base_name': f"{base_name_for_error}"
        }

    return {
        'al_component': al_component, 'nmc_component': nmc_component,
        'nmc_variant': nmc_variant, 'miller_al': miller_al,
        'miller_nmc': miller_nmc, 'base_name': f"{al_component}_{nmc_component}_{nmc_variant}_{miller_nmc}"
    }

def get_input_file_path(interface_filename):
    """統合システムの界面構造ファイルパスを取得"""
    return os.path.join(INPUT_DIR, interface_filename)

def get_compression_file_path(base_name, pressure, comp_temp):
    """圧着シミュレーションファイルのパスを取得"""
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}__P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 界面構造対応の計算条件設定
# =============================================

UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,           # 界面構造用に低速化（1/ps）
    "vacuum_top": 0.0,             # 界面構造用vacuum（Å）
    "temperature": 300.0,          # 引張温度固定（K）
    "max_strain": 1.75,             # 界面用最大ひずみ
    "timestep": 1.0,               # fs
    "elastic_region": (0.005, 0.03), # 界面用弾性領域
    "quality_threshold": 0.85,     # 界面用R²値閾値
    "max_steps": 10000,            # 界面用最大ステップ数
    "output_freq": 50,             # 出力頻度
    "free_region_thickness": 36.0  # 界面構造用固定層厚さ（Å）
}

# 界面構造用早期終了条件
EARLY_STOP_CONDITIONS = {
    "separation_gap": 8.0,         # 界面分離判定の隙間（Å）
    "max_strain_limit": 1.75,       # 最大ひずみ制限
    "min_stress_threshold": -0.5,  # 最小応力閾値（GPa）
    "separation_wait_steps": 300,  # 分離後の待機ステップ
    "stress_drop_ratio": 0.7       # 破断検出用
}

# 界面構造解析対象のパラメータ（サンプリング）
compression_temperatures = [300]      # 圧着シミュレーション温度（K）
compression_pressures = [0.00] # 圧着圧力（GPa）- サンプリング0.001, 0.01
high_temperatures = [500]        # 高温シミュレーション温度（K）- サンプリング
tensile_temperature = 300.0           # 引張シミュレーション温度（K）

# =============================================
# 界面構造対応の固定層設定関数
# =============================================

def identify_interface_region(atoms):
    """界面領域を特定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とNMC原子（Li, Ni, Mn, Co, O）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    nmc_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['Li', 'Ni', 'Mn', 'Co', 'O']]
    
    if not al_indices or not nmc_indices:
        # 界面でない場合のフォールバック
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'nmc_region': (np.mean(z_coords), np.max(z_coords)),
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    nmc_z_coords = z_coords[nmc_indices]
    
    # Al領域とNMC領域の境界を特定
    al_center = np.mean(al_z_coords)
    nmc_center = np.mean(nmc_z_coords)
    
    if al_center < nmc_center:
        # Alが下、NMCが上
        interface_z = (np.max(al_z_coords) + np.min(nmc_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    else:
        print("逆向きです。")# NMCが下、Alが上
        interface_z = (np.max(nmc_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'nmc_region': nmc_region,
        'al_center': al_center,
        'nmc_center': nmc_center,
        'is_interface': True
    }
def set_fixed_atoms_under(atoms,thickness=3.0):
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    indices = [i for i, z in enumerate(z_coords)if z < 3.0]
    # 固定制約の設定
    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        return atoms
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    return atoms
import os
import numpy as np
from ase.io import Trajectory
from ase.constraints import FixAtoms 

# (注意: 以下のコードは 'identify_interface_region(atoms)' 関数が
# 別途定義されていることを前提としています)

def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
    """界面構造で自由領域の厚みを統一する固定原子設定
    (ご提示いただいたコードのまま)
    """
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    total_thickness = z_max - z_min
    
    print(f"自由領域統一固定層設定:")
    print(f"  総材料厚さ: {total_thickness:.2f}Å")
    print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
    # 界面情報を取得 (この関数は別途定義されている必要があります)
    interface_info = identify_interface_region(atoms) 
    
    if interface_info['is_interface']:
        print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
        print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å")
        print(f"  NMC領域: {interface_info['nmc_region'][0]:.2f} - {interface_info['nmc_region'][1]:.2f}Å")
        
        interface_z = interface_info['interface_z']
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = interface_z - free_half_thickness
        free_z_max = interface_z + free_half_thickness
        
        # 材料範囲内に調整
        free_z_min = max(free_z_min, z_min + 2.0)
        free_z_max = min(free_z_max, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
        
    else:
        # 非界面構造の場合
        print(f"  非界面構造: 中心基準で自由領域設定")
        center_z = (z_min + z_max) / 2.0
        free_half_thickness = target_free_thickness / 2.0
        
        free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
        free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
        actual_free_thickness = free_z_max - free_z_min
        
        print(f"  中心位置: {center_z:.2f}Å")
        print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
        print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
        indices = [
            i for i, z in enumerate(z_coords)
            if z < free_z_min or z > free_z_max
        ]
    
    # 構造情報を属性として保存（固定原子がなくても厚さ情報は保存）
    atoms.info['free_region_thickness'] = actual_free_thickness
    atoms.info['free_region_bounds'] = (free_z_min, free_z_max)

    if len(indices) == 0:
        print("  警告: 固定原子がありません")
        atoms.info['fixed_atoms_count'] = 0
        atoms.info['free_atoms_count'] = len(atoms)
        # 既存の制約がクリアされていることを確認
        atoms.set_constraint() 
        return atoms

    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    free_atoms = len(atoms) - len(indices)
    free_ratio = free_atoms / len(atoms)
    fixed_ratio = len(indices) / len(atoms)
    
    print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
    print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
    # 構造情報を属性として保存
    atoms.info['fixed_atoms_count'] = len(indices)
    atoms.info['free_atoms_count'] = free_atoms
    
    return atoms


# def set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness=36.0):
#     """界面構造で自由領域の厚みを統一する固定原子設定
    
#     Parameters:
#     -----------
#     atoms : ase.Atoms
#         対象の原子構造
#     target_free_thickness : float
#         自由領域の目標厚さ（Å）
        
#     Returns:
#     --------
#     atoms : ase.Atoms
#         固定制約が設定された原子構造
#     """
#     positions = atoms.get_positions()
#     z_coords = positions[:, 2]
#     z_min, z_max = np.min(z_coords), np.max(z_coords)
#     total_thickness = z_max - z_min
    
#     print(f"自由領域統一固定層設定:")
#     print(f"  総材料厚さ: {total_thickness:.2f}Å")
#     print(f"  目標自由領域厚さ: {target_free_thickness:.2f}Å")
    
#     # 界面情報を取得
#     interface_info = identify_interface_region(atoms)
    
#     if interface_info['is_interface']:
#         print(f"  界面構造検出: 界面位置 = {interface_info['interface_z']:.2f}Å")
#         print(f"  Al領域: {interface_info['al_region'][0]:.2f} - {interface_info['al_region'][1]:.2f}Å")
#         print(f"  NMC領域: {interface_info['nmc_region'][0]:.2f} - {interface_info['nmc_region'][1]:.2f}Å")
        
#         # 界面中心を基準に自由領域を設定
#         interface_z = interface_info['interface_z']
#         free_half_thickness = target_free_thickness / 2.0
        
#         # 自由領域の範囲
#         free_z_min = interface_z - free_half_thickness
#         free_z_max = interface_z + free_half_thickness
        
#         # 材料範囲内に調整
#         free_z_min = max(free_z_min, z_min + 2.0)  # 最低2Å下は固定
#         free_z_max = min(free_z_max, z_max - 2.0)  # 最低2Å上は固定
        
#         actual_free_thickness = free_z_max - free_z_min
        
#         print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
#         print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
#         # 固定原子の選択（自由領域外を固定）
#         indices = [
#             i for i, z in enumerate(z_coords)
#             if z < free_z_min or z > free_z_max
#         ]
        
#     else:
#         # 非界面構造の場合：中心を基準に自由領域を設定
#         print(f"  非界面構造: 中心基準で自由領域設定")
#         center_z = (z_min + z_max) / 2.0
#         free_half_thickness = target_free_thickness / 2.0
        
#         free_z_min = max(center_z - free_half_thickness, z_min + 2.0)
#         free_z_max = min(center_z + free_half_thickness, z_max - 2.0)
        
#         actual_free_thickness = free_z_max - free_z_min
        
#         print(f"  中心位置: {center_z:.2f}Å")
#         print(f"  自由領域: {free_z_min:.2f} - {free_z_max:.2f}Å")
#         print(f"  実際の自由領域厚さ: {actual_free_thickness:.2f}Å")
        
#         # 固定原子の選択
#         indices = [
#             i for i, z in enumerate(z_coords)
#             if z < free_z_min or z > free_z_max
#         ]
    
#     # 固定制約の設定
#     if len(indices) == 0:
#         print("  警告: 固定原子がありません")
#         return atoms
    
#     constraint = FixAtoms(indices=indices)
#     atoms.set_constraint(constraint)
    
#     # 統計情報
#     free_atoms = len(atoms) - len(indices)
#     free_ratio = free_atoms / len(atoms)
#     fixed_ratio = len(indices) / len(atoms)
    
#     print(f"  固定原子数: {len(indices)}/{len(atoms)} ({fixed_ratio*100:.1f}%)")
#     print(f"  自由原子数: {free_atoms}/{len(atoms)} ({free_ratio*100:.1f}%)")
    
#     # 構造情報を属性として保存
#     atoms.info['free_region_thickness'] = actual_free_thickness
#     atoms.info['free_region_bounds'] = (free_z_min, free_z_max)
#     atoms.info['fixed_atoms_count'] = len(indices)
#     atoms.info['free_atoms_count'] = free_atoms
    
#     return atoms

# =============================================
# 既存の解析クラスとメイン処理（修正版）
# =============================================

class InterfaceComprehensiveTensileAnalyzer(MDExtensionBase):
    """界面構造用改良版引張解析器"""
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 界面特有の解析パラメータ
        self.interface_debonding = False
        self.debonding_step = None
        self.al_nmc_separation = False
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Interface Tensile Analysis\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Interface Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Status Time(ps) InterfaceNotes\n")
    
    def detect_interface_events(self, system, material_strain, stress_zz_GPa):
        """界面特有のイベント検出"""
        
        # Al-NMC界面の分離検出
        if system.current_total_step % 100 == 0:
            atoms = system.ase_atoms
            interface_info = identify_interface_region(atoms)
            
            if interface_info['is_interface']:
                al_region = interface_info['al_region']
                nmc_region = interface_info['nmc_region']
                
                # Al-NMC間の隙間を計算
                if interface_info['al_center'] < interface_info['nmc_center']:
                    gap = nmc_region[0] - al_region[1]
                else:
                    gap = al_region[0] - nmc_region[1]
                
                # 界面剥離の検出
                if gap > EARLY_STOP_CONDITIONS["separation_gap"] and not self.al_nmc_separation:
                    self.al_nmc_separation = True
                    self.debonding_step = system.current_total_step
                    print(f"       🔗 界面剥離検出: Step {self.debonding_step}, 隙間 {gap:.1f}Å")
        
        # その他の検出は既存のロジックを使用
        return ""
    
    def __call__(self, system, integrator):
        # [既存のComprehensiveTensileAnalyzerの__call__メソッドをベースに]
        # [界面特有の解析を追加]
        
        # ひずみ計算
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
        else:
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] / units.GPa
        except:
            stress_zz_GPa = 0.0
        
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        # データ記録
        self.data.append([system.current_total_step, material_strain, stress_zz_GPa, time_ps])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算（界面用に調整されたパラメータ）
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.005 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 15:  # 界面用に閾値調整
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"       ✅ 界面ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # 界面イベント検出
        interface_notes = self.detect_interface_events(system, material_strain, stress_zz_GPa)
        
        # 状態更新
        if self.al_nmc_separation:
            status = "Debonded"
        elif self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # ファイル出力
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {interface_notes}\n")
    
    def get_final_results(self):
        """界面解析結果を返す"""
        results = {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            # 界面特有の結果
            'interface_debonding': self.al_nmc_separation,
            'debonding_step': self.debonding_step,
            'interface_analysis': True
        }
        return results

# =============================================
# メイン処理（統合システム対応版）
# =============================================

def main():
    """統合システム対応のメイン処理"""
    
    print("=" * 80)
    print("NMC界面構造MD解析システム")
    print("統合NMC-Al界面構造生成システムとの連携版")
    print("=" * 80)
    
    # 統合システムで生成された界面構造ファイルを取得
    interface_files = get_interface_files()
    
    if not interface_files:
        print(f"❌ 界面構造ファイルが見つかりません: {INPUT_DIR}")
        print("統合NMC-Al界面構造生成システムを先に実行してください")
        return
    
    print(f"解析対象の界面構造: {len(interface_files)}個")
    
    processed_count = 0
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    for interface_file in interface_files:
        try:
            # ファイル名から材料情報を解析
            material_info = parse_interface_filename(interface_file)
            base_name = material_info['base_name']
            
            print(f"\n{'='*60}")
            print(f"処理開始: {interface_file}")
            print(f"材料: {material_info}")
            print(f"{'='*60}")
            
            input_file = get_input_file_path(interface_file)
            
            # 入力ファイルの存在チェック
            if not os.path.exists(input_file):
                print(f"❌ 入力ファイルが見つかりません: {input_file}")
                continue
            
            # 各パラメータに対してMD計算を実行
            for comp_temp in compression_temperatures:
                for pressure in compression_pressures:
                    for high_temp in high_temperatures:
                        
                        condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                        print(f"\n--- 条件: {condition_name} ---")
                        
                        try:
                            # ファイルパス設定
                            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
                            equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                            high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                            tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                            analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                            summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)

                            # ステップ1: 圧着
                            if not check_md_completed(compression_file, 8000, 100):
                                print("圧着シミュレーション実行中...")
                                # ase_atoms = read(input_file)
                                # set_fixed_atoms_under(ase_atoms)
                                # # ase_atoms = set_interface_fixed_atoms_uniform_free(ase_atoms, UNIFIED_CONDITIONS["free_region_thickness"])
                                # run_interface_compression_md(ase_atoms, pressure, comp_temp, compression_file)
                            else:
                                print("圧着シミュレーションは既に完了しています。")

                            # ステップ2: 平衡化 & 高温処理
                            if not check_md_completed(high_temp_file, 10000, 100):
                                print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                # 平衡化 (NVT, 3ps @ 300K)
                                # atoms_after_comp = read(compression_file, index='-1')
                                atoms_after_comp = read(input_file)
                                set_fixed_atoms_under(atoms_after_comp)
                                # atoms_after_comp = set_interface_fixed_atoms_uniform_free(atoms_after_comp, UNIFIED_CONDITIONS["free_region_thickness"])
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(300.0)
                                integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                                
                                # 高温処理 (NVT, 10ps @ high_temp)
                                atoms_after_equil = read(equil_file, index='-1')
                                set_fixed_atoms_under(atoms_after_equil)
                                atoms_after_equil = set_interface_fixed_atoms_uniform_free(atoms_after_equil, UNIFIED_CONDITIONS["free_region_thickness"])
                                atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                system = ASEMDSystem(atoms)
                                system.init_temperature(float(high_temp))
                                integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                md(system)
                            else:
                                print("高温シミュレーションは既に完了しています。")

                            # ステップ3: 引張試験
                            if not os.path.exists(summary_file):
                                print("引張シミュレーション実行中...")
                                final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                
                                print(f"\n✅ 引張計算完了: {base_name}")
                                print(f"📊 結果サマリー:")
                                print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                            else:
                                print("引張シミュレーションと解析は既に完了しています。")

                        except Exception as e:
                            print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                            import traceback
                            traceback.print_exc()
                            continue

            processed_count += 1
            
        except Exception as e:
            print(f"❌ ファイル {interface_file} の処理でエラー: {e}")
            continue
    
    print(f"\n処理完了: {processed_count}/{len(interface_files)} 個の界面構造を処理しました")


def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        return num_frames >= expected_frames * 0.9
    except Exception:
        return False


class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f}")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)


def rotate_atoms_to_lower(atoms):
    """原子系のZ座標を確認し、全原子を下端（Z=0付近）に移動させる"""
    ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    
    positions = ase_atoms.get_positions()
    if len(positions) == 0:
        return ase_atoms
    z_min = np.min(positions[:, 2])
    
    shift_z = -z_min
    translation = np.array([0, 0, shift_z])
    ase_atoms.translate(translation)
    
    print(f"原子をZ方向に {shift_z:.2f}Å 移動しました")
    
    return ase_atoms

import os
import numpy as np
from ase.io import read, write
from ase.constraints import FixAtoms

# rotate_atoms_to_lower や identify_interface_region, 
# set_interface_fixed_atoms_uniform_free は、
# このファイルと同じ場所にあるか、インポート可能であると仮定します。
# from .utils import rotate_atoms_to_lower, identify_interface_region, set_interface_fixed_atoms_uniform_free

import os
import numpy as np
from ase.io import read, write
from ase.constraints import FixAtoms

# rotate_atoms_to_lower や set_interface_fixed_atoms_uniform_free は、
# このファイルと同じ場所にあるか、インポート可能であると仮定します。
# from .utils import rotate_atoms_to_lower, set_interface_fixed_atoms_uniform_free

def identify_interface_region(atoms):
    """界面領域を特定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とNMC原子（Li, Ni, Mn, Co, O）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    nmc_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['Li', 'Ni', 'Mn', 'Co', 'O']]
    
    if not al_indices or not nmc_indices:
        # 界面でない場合のフォールバック
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'nmc_region': (np.mean(z_coords), np.max(z_coords)),
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    nmc_z_coords = z_coords[nmc_indices]
    
    # Al領域とNMC領域の境界を特定
    al_center = np.mean(al_z_coords)
    nmc_center = np.mean(nmc_z_coords)
    
    if al_center < nmc_center:
        # Alが下、NMCが上
        interface_z = (np.max(al_z_coords) + np.min(nmc_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    else:
        # NMCが下、Alが上
        interface_z = (np.max(nmc_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'nmc_region': nmc_region,
        'al_center': al_center,
        'nmc_center': nmc_center,
        'is_interface': True
    }
import os
import numpy as np
from ase.io import read, write
from ase.constraints import FixAtoms

# rotate_atoms_to_lower や set_interface_fixed_atoms_uniform_free は、
# このファイルと同じ場所にあるか、インポート可能であると仮定します。
# from .utils import rotate_atoms_to_lower, set_interface_fixed_atoms_uniform_free

def identify_interface_region(atoms):
    """界面領域を特定"""
    positions = atoms.get_positions()
    z_coords = positions[:, 2]
    
    # Al原子とNMC原子（Li, Ni, Mn, Co, O）を分離
    al_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Al']
    nmc_indices = [i for i, atom in enumerate(atoms) if atom.symbol in ['Li', 'Ni', 'Mn', 'Co', 'O']]
    
    if not al_indices or not nmc_indices:
        # 界面でない場合のフォールバック
        return {
            'interface_z': np.mean(z_coords),
            'al_region': (np.min(z_coords), np.mean(z_coords)),
            'nmc_region': (np.mean(z_coords), np.max(z_coords)),
            'is_interface': False
        }
    
    al_z_coords = z_coords[al_indices]
    nmc_z_coords = z_coords[nmc_indices]
    
    # Al領域とNMC領域の境界を特定
    al_center = np.mean(al_z_coords)
    nmc_center = np.mean(nmc_z_coords)
    
    if al_center < nmc_center:
        # Alが下、NMCが上
        interface_z = (np.max(al_z_coords) + np.min(nmc_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    else:
        # NMCが下、Alが上
        interface_z = (np.max(nmc_z_coords) + np.min(al_z_coords)) / 2
        al_region = (np.min(al_z_coords), np.max(al_z_coords))
        nmc_region = (np.min(nmc_z_coords), np.max(nmc_z_coords))
    
    return {
        'interface_z': interface_z,
        'al_region': al_region,
        'nmc_region': nmc_region,
        'al_center': al_center,
        'nmc_center': nmc_center,
        'is_interface': True
    }

# def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
#     """
#     界面構造ファイルを読み込み、MD計算用に準備する関数。
#     """
    
#     if not os.path.exists(input_file):
#         raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
#     # --- 1. 構造の読み込みと基本設定 ---
#     atoms = Trajectory(input_file)[-1]
#     atoms.set_constraint() # 既存の拘束をクリア

#     #--- 2. 構造の向きを調整 (Alが下層になるように条件付きで反転) ---
#     al_indices  = [a.index for a in atoms if a.symbol == 'Al']
#     other_indices = [a.index for a in atoms if a.symbol != 'Al']

#     if al_indices and other_indices:
#         al_z_mean = np.mean(atoms.get_positions()[al_indices, 2])
#         other_z_mean = np.mean(atoms.get_positions()[other_indices, 2])

#         if al_z_mean > other_z_mean:
#             print("構造チェック: Al層が上部に検出されたため、構造をZ軸反転します。")
#             positions = atoms.get_positions()
#             positions[:, 2] *= -1
#             atoms.set_positions(positions)
#         else:
#             print("構造チェック: Al層は既に下部にあるため、構造の反転はスキップします。")
#     else:
#         print("構造チェック: Al層またはその他材料が検出できなかったため、反転処理をスキップします。")

#     # --- 3. 構造情報の解析とz_minの0への移動 ---
#     positions = atoms.get_positions()
#     z_min = np.min(positions[:, 2])
    
#     # 構造全体を移動させ、z_minが0になるようにする
#     atoms.translate([0, 0, -z_min])
#     print(f"構造移動: z_min ({z_min:.2f} Å) が 0 Å になるように構造全体を移動しました。")

#     # z座標を再計算
#     z_coords = atoms.get_positions()[:, 2]
#     z_min_new, z_max_new = np.min(z_coords), np.max(z_coords)
#     material_thickness = z_max_new - z_min_new
    
#     print("界面構造解析:")
#     print(f"  材料厚さ: {material_thickness:.2f} Å")
#     print(f"  原子数: {len(atoms)}")

#     # --- 4. セルと真空層の設定 ---
#     vacuum_to_add = 1.0
#     print(f"設定: 構造上部に {vacuum_to_add} Å の真空層を追加します。")
    
#     # セルサイズを更新
#     new_cell_z = material_thickness + vacuum_to_add
#     cell = atoms.get_cell()
#     cell[2, 2] = new_cell_z
#     atoms.set_cell(cell, scale_atoms=False) # 原子は既に正しい位置なのでスケールしない

#     strain_reference_length = material_thickness # 基準長は材料厚さ
#     strain_method = "material_length"

#     # --- 5. 自由領域・固定領域の設定 ---
#     atoms = set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness) # この関数の定義が必要です

#     # --- 6. 構造情報を `atoms.info` に集約 ---
#     interface_info = identify_interface_region(atoms)
#     ase_atoms=atoms.copy()
#     initial_reference = ase_atoms.get_cell()[2, 2]
#     total_cell_z = atoms.get_cell()[2, 2]
#     atoms.info = {
#         "input_file": os.path.basename(input_file),
#         "material_thickness": material_thickness,
#         "vacuum_top": vacuum_to_add,
#         "total_cell_z": total_cell_z,
#         "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
#         "strain_reference_length": strain_reference_length,
#         "strain_method": strain_method,
#         "interface_info": interface_info,
#         'set_interface_fixed_atoms_uniform_free' が返す情報をここに追加
#     }
#     ase_atoms.info = {
#         "input_file": os.path.basename(input_file),
#         "material_thickness": material_thickness,
#         "vacuum_top": vacuum_to_add,
#         "total_cell_z": total_cell_z,
#         "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
#         "strain_reference_length": strain_reference_length,
#         "strain_method": strain_method,
#         "interface_info": interface_info,
#         # 'set_interface_fixed_atoms_uniform_free' が返す情報をここに追加
#     }
        
#     structure_info = {
#         "material_thickness": material_thickness,
#         "vacuum_top": vacuum_top,
#         "total_cell_z": new_cell_z,
#         "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
#         "interface_info": interface_info,
#         "strain_method": "material_length" if vacuum_top > 0 else "cell_length",
#         "free_region_thickness": ase_atoms.info.get('free_region_thickness', target_free_thickness),
#         "free_region_bounds": ase_atoms.info.get('free_region_bounds', (0, target_free_thickness)),
#         "fixed_atoms_count": ase_atoms.info.get('fixed_atoms_count', 0),
#         "free_atoms_count": ase_atoms.info.get('free_atoms_count', 0)
#     }
    
#     print("準備完了:")
#     print(f"  最終セルZ長: {atoms.info['total_cell_z']:.2f} Å")
#     print(f"  材料占有率: {atoms.info['material_ratio']*100:.1f}%")
#     if atoms.info['interface_info']['is_interface']:
#         print(f"  界面検出: Yes (Z ≈ {atoms.info['interface_info']['interface_z']:.2f} Å)")
#     else:
#         print(f"  界面検出: No")
    
#     return atoms,initial_reference,structure_info

def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
    """
    界面構造ファイルを読み込み、MD計算用に準備する関数。
    (末尾の info 格納処理を修正)
    """
    
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
    # --- 1. 構造の読み込みと基本設定 ---
    atoms = Trajectory(input_file)[-1]
    atoms.set_constraint() # 既存の拘束をクリア

    #--- 2. 構造の向きを調整 (Alが下層になるように条件付きで反転) ---
    al_indices  = [a.index for a in atoms if a.symbol == 'Al']
    other_indices = [a.index for a in atoms if a.symbol != 'Al']

    if al_indices and other_indices:
        al_z_mean = np.mean(atoms.get_positions()[al_indices, 2])
        other_z_mean = np.mean(atoms.get_positions()[other_indices, 2])

        if al_z_mean > other_z_mean:
            print("構造チェック: Al層が上部に検出されたため、構造をZ軸反転します。")
            positions = atoms.get_positions()
            positions[:, 2] *= -1
            atoms.set_positions(positions)
        else:
            print("構造チェック: Al層は既に下部にあるため、構造の反転はスキップします。")
    else:
        print("構造チェック: Al層またはその他材料が検出できなかったため、反転処理をスキップします。")

    # --- 3. 構造情報の解析とz_minの0への移動 ---
    positions = atoms.get_positions()
    z_min = np.min(positions[:, 2])
    
    atoms.translate([0, 0, -z_min])
    print(f"構造移動: z_min ({z_min:.2f} Å) が 0 Å になるように構造全体を移動しました。")

    z_coords = atoms.get_positions()[:, 2]
    z_min_new, z_max_new = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max_new - z_min_new
    
    print("界面構造解析:")
    print(f"  材料厚さ: {material_thickness:.2f} Å")
    print(f"  原子数: {len(atoms)}")

    # --- 4. セルと真空層の設定 ---
    # 元のコードでは vacuum_top 引数を使わず 1.0 をハードコードしていたため、
    # その挙動を維持します。
    vacuum_to_add = 1.0 
    print(f"設定: 構造上部に {vacuum_to_add} Å の真空層を追加します。")
    
    new_cell_z = material_thickness + vacuum_to_add
    cell = atoms.get_cell()
    cell[2, 2] = new_cell_z
    atoms.set_cell(cell, scale_atoms=False) 

    # 元のコードのロジックを維持
    strain_reference_length = material_thickness
    strain_method = "material_length"

    # --- 5. 自由領域・固定領域の設定 ---
    # この時点で atoms.info に 'free_region_thickness' などが設定される
    atoms = set_interface_fixed_atoms_uniform_free(atoms, target_free_thickness) 

    # --- 6. 構造情報を `atoms.info` に集約 ---
    interface_info = identify_interface_region(atoms) # 情報を再取得
    initial_reference = atoms.get_cell()[2, 2]
    total_cell_z = atoms.get_cell()[2, 2]

    # atoms.info を更新 (set_interface_fixed... で追加された情報以外を update)
    atoms.info.update({
        "input_file": os.path.basename(input_file),
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": total_cell_z,
        "material_ratio": material_thickness / total_cell_z if total_cell_z > 0 else 0,
        "strain_reference_length": strain_reference_length,
        "strain_method": strain_method,
        "interface_info": interface_info,
        # 'free_region_thickness' などは set_interface... 関数内で既に atoms.info に追加されている
    })
    
    # ★修正：不要な ase_atoms.copy() と ase_atoms.info の設定を削除
    
    # 呼び出し元に返す structure_info 辞書を作成
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_to_add,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
        "interface_info": interface_info,
        "strain_method": strain_method,
        
        # set_interface_fixed... で設定された情報を atoms.info から取得
        "free_region_thickness": atoms.info.get('free_region_thickness', None),
        "free_region_bounds": atoms.info.get('free_region_bounds', None),
        "fixed_atoms_count": atoms.info.get('fixed_atoms_count', 0),
        "free_atoms_count": atoms.info.get('free_atoms_count', len(atoms))
    }
    
    print("準備完了:")
    print(f"  最終セルZ長: {atoms.info['total_cell_z']:.2f} Å")
    print(f"  材料占有率: {atoms.info['material_ratio']*100:.1f}%")
    if atoms.info['interface_info'].get('is_interface', False):
        print(f"  界面検出: Yes (Z ≈ {atoms.info['interface_info'].get('interface_z', 0.0):.2f} Å)")
    else:
        print(f"  界面検出: No")
    
    return atoms, initial_reference, structure_info
    
# def prepare_interface_structure_for_md(input_file, vacuum_top, target_free_thickness):
#     """界面構造をMD計算用に準備（自由領域厚み統一版）"""
    
#     if not os.path.exists(input_file):
#         raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
    
#     # CIFファイルの読み込み
#     ase_atoms = read(input_file, index='-1') # Read the last frame if it's a trajectory
#     position = ase_atoms.get_positions()
#     ase_atoms.set_constraint()
    
#     # position[:, 2] *= -1
#     ase_atoms.set_positions(position)
#     ase_atoms = rotate_atoms_to_lower(ase_atoms)
#     # path_before="/home/jovyan/Kaori/MD/LiB_2/structure/MD4/md_before.xyz"
#     # write(str(path_before), ase_atoms)
#     # 構造解析
#     positions = ase_atoms.get_positions()
#     z_coords = positions[:, 2]
#     z_min, z_max = np.min(z_coords), np.max(z_coords)
#     material_thickness = z_max - z_min
    
#     print(f"界面構造解析:")
#     print(f"  材料厚さ: {material_thickness:.2f}Å")
#     print(f"  原子数: {len(ase_atoms)}")
    
#     # 界面情報取得
#     interface_info = identify_interface_region(ase_atoms)
    
#     new_cell_z = material_thickness
#     if vacuum_top > 0:
#         print(f"設定: vacuum有り ({vacuum_top}Å)")
#         add_vacuum(ase_atoms, vacuum=vacuum_top, axis=2)
#         new_cell_z += vacuum_top
#         initial_reference = material_thickness
#     else:
#         print(f"設定: vacuum無し")
#         initial_reference = ase_atoms.get_cell()[2, 2]

#     # 自由領域厚み統一の固定層設定
#     ase_atoms = set_interface_fixed_atoms_uniform_free(ase_atoms, target_free_thickness)
    
#     structure_info = {
#         "material_thickness": material_thickness,
#         "vacuum_top": vacuum_top,
#         "total_cell_z": new_cell_z,
#         "material_ratio": material_thickness / new_cell_z if new_cell_z > 0 else 0,
#         "interface_info": interface_info,
#         "strain_method": "material_length" if vacuum_top > 0 else "cell_length",
#         "free_region_thickness": ase_atoms.info.get('free_region_thickness', target_free_thickness),
#         "free_region_bounds": ase_atoms.info.get('free_region_bounds', (0, target_free_thickness)),
#         "fixed_atoms_count": ase_atoms.info.get('fixed_atoms_count', 0),
#         "free_atoms_count": ase_atoms.info.get('free_atoms_count', 0)
#     }
    
#     print(f"準備完了:")
#     print(f"  セルZ長: {ase_atoms.get_cell()[2,2]:.2f}Å")
#     print(f"  材料占有率: {structure_info['material_ratio']*100:.1f}%")
#     print(f"  界面検出: {interface_info['is_interface']}")
#     print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    
#     return ase_atoms, initial_reference, structure_info


def run_interface_compression_md(ase_atoms, pressure, comp_temp, output_file):
    """界面構造用圧着MD"""
    
    ase_atoms.set_pbc(True)
    ase_atoms.wrap()
    # optimizer = BFGS(ase_atoms, trajectory='optimization.traj')
    # optimizer.run(fmax=0.05)
    # atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    
    press_ase = pressure * units.GPa
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    
    system = ASEMDSystem(atoms)
    system.init_temperature(comp_temp)
    
    # 初期NVT平衡化
    nvt_integrator = LangevinIntegrator(timestep=1.0, temperature=comp_temp)
    md = MDFeature(nvt_integrator, n_run=1000, estimator_fn=estimator_fn)
    md(system, extensions=[(PrintCellShape(), 200)])
    
    # NPT圧着
    integrator = NPTIntegrator(
        timestep=1.0,
        temperature=comp_temp,
        pressure=press_ase,
        ttime=20*units.fs,
        pfactor=2e6*units.GPa*(units.fs**2),
        mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
    )
    # pfactor=2e6*units.GPa*(units.fs**2),
    
    md = MDFeature(
        integrator, 
        n_run=8000, 
        traj_file_name=output_file,
        traj_freq=100, 
        estimator_fn=estimator_fn
    )
    
    md(system, extensions=[(PrintCellShape(), 500)])
    return output_file


def run_interface_tensile_md(input_file, output_file, analysis_file):
    """界面構造用引張MD（自由領域厚み統一版）"""
    
    # 構造準備
    
    ase_atoms,initial_reference,structure_info = prepare_interface_structure_for_md(
        input_file, 
        UNIFIED_CONDITIONS["vacuum_top"],
        UNIFIED_CONDITIONS["free_region_thickness"]
    )
    
    # MD設定
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    system = ASEMDSystem(atoms)
    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
    
    # 引張設定
    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
    max_strain = UNIFIED_CONDITIONS["max_strain"]
    timestep = UNIFIED_CONDITIONS["timestep"]
    
    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
    
    if initial_reference == 0:
        initial_reference = 50 # Fallback value to avoid division by zero
    
    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
    else:
        final_cell_length = initial_reference * (1 + max_strain)
    
    latt = np.array(system.ase_atoms.cell)
    latt[2, 2] = final_cell_length
    
    print(f"自由領域統一引張設定:")
    print(f"  統一自由領域厚さ: {structure_info['free_region_thickness']:.2f}Å")
    print(f"  自由原子数: {structure_info['free_atoms_count']}")
    print(f"  固定原子数: {structure_info['fixed_atoms_count']}")
    print(f"  引張ひずみ速度: {strain_rate} 1/ps")

    # 解析器セットアップ
    analyzer = InterfaceComprehensiveTensileAnalyzer(
        initial_reference, structure_info, UNIFIED_CONDITIONS, input_file, analysis_file
    )
    
    # MD実行
    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
    deform = DeformScheduler(latt, total_steps)
    info = PrintCellShape()
    
    estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
    md = MDFeature(
        integrator,
        n_run=total_steps,
        traj_file_name=output_file,
        traj_freq=UNIFIED_CONDITIONS["output_freq"],
        estimator_fn=estimator_fn
    )
    
    extensions = [
        (info, 100),
        (deform, 1),
        (analyzer, 10)
    ]
    
    md(system, extensions=extensions)
    
    return analyzer.get_final_results()


def save_interface_results(results, summary_file, base_name, material_info, pressure, comp_temp, high_temp):
    """界面構造解析結果の保存（自由領域統一版）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"NMC-Al界面構造MD解析結果 - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write(f"界面構造情報:\n")
        f.write(f"  Al成分: {material_info['al_component']}\n")
        f.write(f"  NMC成分: {material_info['nmc_component']}\n")
        f.write(f"  NMC組成バリアント: {material_info['nmc_variant']}\n")
        f.write(f"  Al Miller指数: {material_info['miller_al']}\n")
        f.write(f"  NMC Miller指数: {material_info['miller_nmc']}\n\n")
        
        f.write(f"統一自由領域設定:\n")
        structure_info = results['structure_info']
        free_count = structure_info.get('free_atoms_count', 0)
        fixed_count = structure_info.get('fixed_atoms_count', 1) # Avoid division by zero
        total_atoms = free_count + fixed_count
        free_ratio = free_count / total_atoms if total_atoms > 0 else 0

        f.write(f"  目標自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']:.1f}Å\n")
        f.write(f"  実際の自由領域厚さ: {structure_info.get('free_region_thickness', 0):.1f}Å\n")
        f.write(f"  自由領域範囲: {structure_info.get('free_region_bounds', (0, 0))[0]:.1f} - {structure_info.get('free_region_bounds', (0, 0))[1]:.1f}Å\n")
        f.write(f"  自由原子数: {free_count}\n")
        f.write(f"  固定原子数: {fixed_count}\n")
        f.write(f"  自由原子比率: {free_ratio * 100:.1f}%\n\n")
        
        f.write(f"計算条件:\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps\n\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        f.write(f"界面特性:\n")
        if results.get('interface_debonding', False):
            f.write(f"  界面剥離: Step {results.get('debonding_step', 'N/A')}\n")
        else:
            f.write(f"  界面剥離: なし\n")
        
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']}\n")
        else:
            f.write(f"  破断検出: なし\n")
        
        f.write(f"\n品質評価:\n")
        if results['young_r2'] > 0.9:
            quality = "Excellent"
        elif results['young_r2'] > 0.8:
            quality = "Good"
        elif results['young_r2'] > 0.7:
            quality = "Fair"
        else:
            quality = "Poor"
            
        f.write(f"  データ品質: {quality} (R² = {results['young_r2']:.3f})\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        
        f.write(f"\n自由領域統一システムの利点:\n")
        f.write(f"  統一自由領域厚さ: {UNIFIED_CONDITIONS['free_region_thickness']}Å\n")
        f.write(f"  公平な機械特性比較が可能\n")
        f.write(f"  界面中心を基準とした適応的固定層設定\n")
        f.write(f"  材料厚さの違いを考慮した統一評価\n")
        f.write(f"  計算効率と比較性の両立\n")

# 使用例とテスト関数
def test_uniform_free_region():
    """自由領域統一システムのテスト関数"""
    print("自由領域統一システムのテスト開始")
    print("-" * 50)
    
    # テスト用の仮想界面構造を作成
    from ase import Atoms
    
    # Al部分（下部）
    al_positions = []
    for z in np.arange(0, 10, 2):
        for x in np.arange(0, 6, 2):
            for y in np.arange(0, 6, 2):
                al_positions.append([x, y, z])
    
    # NMC部分（上部）
    nmc_positions = []
    elements = ['Li', 'Ni', 'Mn', 'Co', 'O']
    for z in np.arange(12, 25, 2):
        for x in np.arange(0, 6, 2):
            for y in np.arange(0, 6, 2):
                nmc_positions.append([x, y, z])
    
    # 原子構造作成
    positions = al_positions + nmc_positions
    symbols = ['Al'] * len(al_positions) + elements * (len(nmc_positions) // len(elements) + 1)
    symbols = symbols[:len(positions)]
    
    test_atoms = Atoms(symbols=symbols, positions=positions)
    test_atoms.set_cell([8, 8, 30])
    test_atoms.set_pbc(True)
    
    print(f"テスト構造: {len(test_atoms)} atoms, 厚さ {np.max(test_atoms.positions[:, 2]) - np.min(test_atoms.positions[:, 2]):.1f}Å")
    
    # 自由領域統一固定の適用
    target_free = 36.0
    result_atoms = set_interface_fixed_atoms_uniform_free(test_atoms, target_free)
    
    # 結果確認
    print(f"\n結果:")
    print(f"  目標自由領域厚さ: {target_free}Å")
    print(f"  実際の自由領域厚さ: {result_atoms.info.get('free_region_thickness', 'N/A')}Å")
    print(f"  自由/固定比率: {result_atoms.info.get('free_atoms_count', 0)}/{result_atoms.info.get('fixed_atoms_count', 0)}")
    
    return result_atoms


def create_comparison_report(md_results_dir):
    """自由領域統一による結果比較レポートを作成"""
    
    if not os.path.exists(md_results_dir):
        print(f"MD結果ディレクトリが見つかりません: {md_results_dir}")
        return
    
    summary_files = list(Path(md_results_dir).glob("tensile/*_summary.txt"))
    if not summary_files:
        print("解析結果ファイルが見つかりません")
        return
    
    print(f"自由領域統一比較レポート作成中...")
    
    results_data = []
    for summary_file in summary_files:
        try:
            with open(summary_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # 基本情報の抽出
            result = {'filename': summary_file.stem}
            
            # 自由領域情報の抽出
            if '実際の自由領域厚さ:' in content:
                line = [l for l in content.split('\n') if '実際の自由領域厚さ:' in l][0]
                result['actual_free_thickness'] = float(line.split(':')[1].split('Å')[0].strip())
            
            if '自由原子数:' in content:
                line = [l for l in content.split('\n') if '自由原子数:' in l][0]
                result['free_atoms'] = int(line.split(':')[1].strip())
            
            if '固定原子数:' in content:
                line = [l for l in content.split('\n') if '固定原子数:' in l][0]
                result['fixed_atoms'] = int(line.split(':')[1].strip())
            
            # 機械特性の抽出
            if 'ヤング率:' in content:
                line = [l for l in content.split('\n') if 'ヤング率:' in l][0]
                result['young_modulus'] = float(line.split(':')[1].split('GPa')[0].strip())
            
            results_data.append(result)
            
        except Exception as e:
            print(f"ファイル処理エラー {summary_file.name}: {e}")
    
    # 統計レポート
    if results_data:
        df = pd.DataFrame(results_data)
        
        report_path = Path(md_results_dir).parent / "free_region_comparison_report.txt"
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("自由領域統一システム比較レポート\n")
            f.write("=" * 50 + "\n")
            f.write(f"作成日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            if 'actual_free_thickness' in df.columns:
                f.write(f"自由領域厚さ統計:\n")
                f.write(f"  平均: {df['actual_free_thickness'].mean():.1f}Å\n")
                f.write(f"  標準偏差: {df['actual_free_thickness'].std():.1f}Å\n")
                f.write(f"  範囲: {df['actual_free_thickness'].min():.1f} - {df['actual_free_thickness'].max():.1f}Å\n\n")
            
            if 'free_atoms' in df.columns and 'fixed_atoms' in df.columns:
                df['free_ratio'] = df['free_atoms'] / (df['free_atoms'] + df['fixed_atoms'])
                f.write(f"自由原子比率統計:\n")
                f.write(f"  平均: {df['free_ratio'].mean()*100:.1f}%\n")
                f.write(f"  標準偏差: {df['free_ratio'].std()*100:.1f}%\n")
                f.write(f"  範囲: {df['free_ratio'].min()*100:.1f} - {df['free_ratio'].max()*100:.1f}%\n\n")
            
            f.write(f"統一性評価:\n")
            if 'actual_free_thickness' in df.columns and df['actual_free_thickness'].mean() != 0:
                cv_thickness = df['actual_free_thickness'].std() / df['actual_free_thickness'].mean()
                f.write(f"  自由領域厚さの変動係数: {cv_thickness:.3f}\n")
                if cv_thickness < 0.1:
                    f.write(f"  → 非常に良い統一性\n")
                elif cv_thickness < 0.2:
                    f.write(f"  → 良い統一性\n")
                else:
                    f.write(f"  → 改善の余地あり\n")
        
        print(f"比較レポートを作成しました: {report_path}")
    
    return results_data

# if __name__ == "__main__":
#     main()

/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of pfp-api-client is available. Please consider upgrading by `pip install -U pfp-api-client`.
  warnings.warn(


/home/jovyan/.py311/lib/python3.11/site-packages/pfp_api_client/__init__.py:42: UserWarning: New version of matlantis-features is available. Please consider upgrading by `pip install -U matlantis-features`.
  warnings.warn(


In [2]:
import os
import time  # time.sleep() を使うためにインポート
# ... (他のimport文は省略) ...
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode
MATLANTIS_AVAILABLE = True
print("✅ Matlantis (matlantis_features) モジュールを正常にインポートしました。")
import numpy as np
# (from ase.io import read など、他のimport文)

def find_overlapping_atoms(atoms, threshold=0.8):
    """
    原子同士が近すぎる(重なっている)ペアを検出し、削除対象のインデックスを返す
    
    :param atoms: ASE Atoms object
    :param threshold: これより近い距離(Å)を「重なり」とみなす
    :return: (set) 削除対象となる原子のインデックス
    """
    print(f"  [Check] 原子間の重なりをチェック (閾値 = {threshold} Å)...")
    
    # 全ペアの距離行列を取得 (mic=Trueで周期境界を考慮)
    distances = atoms.get_all_distances(mic=True)
    
    # 対角成分(自分自身との距離)を無限大にして、検索対象から除外
    np.fill_diagonal(distances, np.inf)
    
    # 閾値以下のペアを検索 (row_indices, col_indices)
    overlapping_pairs = np.where(distances < threshold)
    
    if len(overlapping_pairs[0]) == 0:
        print("  [Check] ✔️ 重なっている原子は見つかりませんでした。")
        return set() # 空のセット

    # 削除するインデックスをセットで管理 (重複削除のため)
    indices_to_delete = set()
    
    rows, cols = overlapping_pairs
    for i, j in zip(rows, cols):
        # (i, j) と (j, i) の重複カウントを防ぐため i < j のみ処理
        if i < j: 
            dist = distances[i, j]
            print(f"  [Check] 🚨 重複を発見: Atom {i} ({atoms[i].symbol}) と Atom {j} ({atoms[j].symbol})")
            print(f"          距離: {dist:.4f} Å")
            
            # ペアの片方(インデックスが大きい方)を削除リストに追加
            indices_to_delete.add(j)

    print(f"  [Check] ⚠️ {len(indices_to_delete)} 個の重複原子を削除対象とします。")
    return indices_to_delete

def remove_atoms_by_index(atoms, indices):
    """
    インデックスのリストに基づいて原子を削除する
    (インデックスがずれないよう、降順で削除する)
    """
    if not indices:
        return atoms

    # インデックスを降順にソート
    sorted_indices = sorted(list(indices), reverse=True)
    
    # del atoms[index] を使って削除
    for idx in sorted_indices:
        del atoms[idx]
            
    print(f"  [Check] {len(sorted_indices)} 個の原子を削除しました。")
    return atoms

from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator, NVTBerendsenIntegrator
def run_constant_temp_md(md_params):
    """一定温度でNVT-MDを実行するためのシンプルな関数"""
    estimator_fn = pfp_estimator_fn(
        model_version=md_params['model_version'],
        calc_mode=md_params['calc_mode'],
    )
    system = ASEMDSystem(
        atoms=md_params['atoms'],
        step=0,
        time=0.0,
    )
    integrator = NVTBerendsenIntegrator(
        timestep=md_params['timestep'],
        temperature=md_params['temperature'],  # スケジューラではなく固定温度を指定
        taut=100.,
        fixcm=True,
    )
    # integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
    md = MDFeature(
        integrator=integrator,
        n_run=md_params['n_run'],
        show_progress_bar=True,
        show_logger=False,
        logger_interval=md_params['logger_interval'],
        estimator_fn=estimator_fn,
        traj_file_name=f"{md_params['traj_filepath']}",
        traj_freq=md_params['traj_freq'],
    )

    # print(f"\n--- 追加計算を開始: {md_params['fname']} ---")
    print(f"ステップ数: {md_params['n_run']} ({md_params['n_run'] * md_params['timestep']/1000:.1f} ps), 維持温度: {md_params['temperature']:.2f} K")

    md_results = md(
        system=system,
        # extensions=[
        #     (PrintWriteLog(fname=md_params['fname'], dirout=md_params['dirout'], stdout=True), md_params['logger_interval'])
        # ]
    )
    # print(f"--- 追加計算が完了: {md_params['fname']} ---")
    return md_results

def main():
    """統合システム対応のメイン処理 (最大40件まで処理)"""

    print("=" * 80)
    print("NMC界面構造MD解析システム (最大40件処理モード)")
    print("=" * 80)

    # --- 処理件数の設定 ---
    MAX_FILES_TO_PROCESS = 60  # 終了するファイル件数
    processed_files = set()    # 処理済みのファイル名を記録するセット
    
    # --- 2. 処理済み件数が 40件 に達するまでループ ---
    while len(processed_files) < MAX_FILES_TO_PROCESS:
        try:
            print(f"\n{'='*30} ファイルチェック開始 ({time.strftime('%Y-%m-%d %H:%M:%S')}) {'='*30}")
            print(f"(現在 {len(processed_files)} / {MAX_FILES_TO_PROCESS} 件 処理済み)")
            
            # 現在のファイルリストを取得
            current_files = get_interface_files()
            
            # 新規ファイルを特定 (現在のリスト - 処理済みリスト)
            new_files = set(current_files) - processed_files

            if not new_files:
                print("新しい解析対象ファイルはありません。30秒間待機します...")
                time.sleep(30)  # 30秒待機
                continue  # ループの最初に戻る

            print(f"✨ {len(new_files)}個の新しいファイルを発見。処理を開始します。")
            
            # 新しく見つかったファイルだけをループ処理
            for interface_file in sorted(list(new_files)):
                
                # --- ループの最初で処理件数を再チェック ---
                # (複数のファイルが一度に見つかった場合、上限を超えないようにするため)
                if len(processed_files) >= MAX_FILES_TO_PROCESS:
                    print(f"\n上限の {MAX_FILES_TO_PROCESS} 件に達したため、新規ファイルの処理を停止します。")
                    break  # この 'for' ループを抜ける

                try:
                    #
                    # (元々の処理ロジック)
                    #
                    material_info = parse_interface_filename(interface_file)
                    base_name = material_info['base_name']
                    
                    print(f"\n--- 処理開始: {interface_file} ---")
                    
                    input_file = get_input_file_path(interface_file)
                    
                    if not os.path.exists(input_file):
                        print(f"❌ 入力ファイルが見つかりません: {input_file}")
                        processed_files.add(interface_file) # エラーでも処理済みリストに追加
                        print(f"    (処理済みカウント: {len(processed_files)}/{MAX_FILES_TO_PROCESS})")
                        continue
                    
                    # 各パラメータに対してMD計算を実行
                    for comp_temp in compression_temperatures:
                    #     for pressure in compression_pressures:
                    #    for comp_temp in compression_temperatures:
                        for pressure in compression_pressures:
                            for high_temp in high_temperatures:
                                
                                condition_name = f"P{pressure:.3f}_T{comp_temp}K_HT{high_temp}K"
                                print(f"\n--- 条件: {condition_name} ---")
                                
                                try:
                                    # ファイルパス設定
                                    compression_file = get_compression_file_path(base_name, pressure, comp_temp)
                                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                                    high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                                    analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                                    summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
        
                                    # ステップ1: 圧着
                                    if not check_md_completed(compression_file, 8000, 100):
                                        print("圧着シミュレーション実行中...")
                                        # ase_atoms = read(input_file)
                                        # set_fixed_atoms_under(ase_atoms)
                                        # # ase_atoms = set_interface_fixed_atoms_uniform_free(ase_atoms, UNIFIED_CONDITIONS["free_region_thickness"])
                                        # run_interface_compression_md(ase_atoms, pressure, comp_temp, compression_file)
                                    else:
                                        print("圧着シミュレーションは既に完了しています。")
        
                                    # ステップ2: 平衡化 & 高温処理
                                    if not check_md_completed(high_temp_file, 10000, 100):
                                        print(f"平衡化と高温シミュレーション({high_temp}K)を実行中...")
                                        # 平衡化 (NVT, 3ps @ 300K)
                                        # atoms_after_comp = read(compression_file, index='-1')
                                        opt_dir = os.path.join(OUTPUT_DIR, "opt")
                                    
                                        # 2. ディレクトリが存在しない場合に作成 (存在してもエラーにならない)
                                        os.makedirs(opt_dir, exist_ok=True) 
                                        
                                        # 3. 最終的なファイルパスを構築
                                        # (例: "results/opt/base_name_optimized.xyz")
                                        base_filename = f"{base_name}_optimized.xyz"
                                        base_trajname = f"{base_name}_optimized.traj"
                                        output_filepath = os.path.join(opt_dir, base_filename)
                                        opt_trajpath = os.path.join(opt_dir,base_trajname)
                                        if not os.path.exists(opt_trajpath):
                                            initial_atoms = read(input_file)
                                        # overlapping_indices = find_overlapping_atoms(initial_atoms, threshold=0.8) 
                                        
                                        # if overlapping_indices:
                                        #     # 削除関数を呼び出して initial_atoms を更新
                                        #     initial_atoms = remove_atoms_by_index(initial_atoms, overlapping_indices)
                                        #     print(f"  [2-1b] ✔️ 重複削除後の原子数: {len(initial_atoms)}") 
                                            
                                            print(initial_atoms.get_chemical_symbols)
                                            positions = initial_atoms.get_positions()
                                            
                                            x_min, x_max = np.min(positions[:, 0]), np.max(positions[:, 0])
                                            y_min, y_max = np.min(positions[:, 1]), np.max(positions[:, 1])
                                            z_min, z_max = np.min(positions[:, 2]), np.max(positions[:, 2])
                                            material_thickness = z_max - z_min
        
                                            # 原子をX=0, Y=0にもシフト
                                            initial_atoms.translate([-x_min, -y_min,  -z_min])
                                            
                                            # 新しいセルを作成
                                            new_cell = np.array([x_max - x_min, y_max - y_min, material_thickness + 1.0] # Zベクトル (真空層1A)
                                            )
                                            
                                            cell = initial_atoms.get_cell()
                                            # Z方向のセル長を原子の厚み + 真空層 (例: 1.0A) に設定
                                            cell[2, 2] = material_thickness + 1.0 
                                            initial_atoms.set_cell(new_cell, scale_atoms=False)
                                            
                                            # PBC設定とセンタリング
                                            initial_atoms.set_pbc([True, True, False])
                                            initial_atoms.wrap()
                                            initial_atoms.center(vacuum=0, axis=2) # Z方向の中央揃え (PBC=Falseのため)
                                            print(f"  [2-1] ✔️ セル設定完了 (PBC: {initial_atoms.pbc})")
        
                                            # --- 2-2. 固定層の設定 ---
                                            print("  [2-2] 固定層を設定...")
                                            set_fixed_atoms_under(initial_atoms)
                                            # initial_atoms = set_interface_fixed_atoms_uniform_free(initial_atoms, UNIFIED_CONDITIONS["free_region_thickness"])
                                            
                                            # --- 2-3. 構造最適化 (Matlantis) ---
                                            print(f"  [2-3] 構造最適化 (Matlantis v7.0.0) を開始...")
                                            optimized_structure = None
                                            try:
                                                # MatlantisAtoms に変換 (固定層情報も引き継がれる)
                                                matlantis_atoms = MatlantisAtoms.from_ase_atoms(initial_atoms)
        
                                                # 最適化用のEstimator関数を定義 (stressは不要)
                                                estimator_for_opt = pfp_estimator_fn(
                                                    model_version='v7.0.0',
                                                    calc_mode='crystal_u0_plus_d3'
                                            
                                                )
        
                                                # オプティマイザを初期化
                                                matlantis_optimizer = FireLBFGSASEOptFeature(
                                                    estimator_fn=estimator_for_opt,
                                                    n_run=3000,
                                                    fmax=0.01,
                                                    show_progress_bar=True, 
                                                    trajectory=opt_trajpath
                                                )
        
                                                # 最適化を実行
                                                result = matlantis_optimizer(matlantis_atoms)
                                                
                                                # 結果をASE Atomsに戻す
                                                optimized_structure = result.atoms.ase_atoms
                                                opt_dir = os.path.join(OUTPUT_DIR, "opt")
                                            
                                                # 2. ディレクトリが存在しない場合に作成 (存在してもエラーにならない)
                                                os.makedirs(opt_dir, exist_ok=True) 
                                                
                                                # 3. 最終的なファイルパスを構築
                                                # (例: "results/opt/base_name_optimized.xyz")
                                                base_filename = f"{base_name}_optimized.xyz"
                                                output_filepath = os.path.join(opt_dir, base_filename)
                                                
                                                # 4. write関数に完全なファイルパスを渡す
                                                write(output_filepath, optimized_structure)
                                                final_energy = result.output.energy_log[-1]
                                                print(f"  [2-3] ✔️ Matlantis最適化完了。最終エネルギー: {final_energy:.3f} eV")

                                            except Exception as e:
                                                print(f"*** 🚨 エラー: Matlantisでの最適化中に問題が発生しました: {e} ***")
                                                import traceback
                                                traceback.print_exc()
                                                print("      最適化されていない構造でMDを続行します。")
                                                optimized_structure = initial_atoms # エラー時は最適化前の構造を使う
                                                
                                    
                                        atoms_after_comp=Trajectory(opt_trajpath)[-1]
                                        if  atoms_after_comp.calc is not None:
                                            print("    古い計算機 (.calc) をリセットします。")
                                            # atoms_after_comp.calc = 
                                        positions = atoms_after_comp.get_positions()
                                        # atoms_after_comp.caLc=
                                        x_min, x_max = np.min(positions[:, 0]), np.max(positions[:, 0])
                                        y_min, y_max = np.min(positions[:, 1]), np.max(positions[:, 1])
                                        z_min, z_max = np.min(positions[:, 2]), np.max(positions[:, 2])
                                        material_thickness = z_max - z_min
    
                                        # 原子をX=0, Y=0にもシフト
                                        atoms_after_comp.translate([-x_min, -y_min,  -z_min])
                                        
                                        # 新しいセルを作成
                                        new_cell = np.array([x_max - x_min, y_max - y_min, material_thickness + 1.0] # Zベクトル (真空層1A)
                                        )
                                        
                                        atoms_after_comp.set_cell(new_cell,scale_atoms=False)
                                        atoms_after_comp.wrap()
                                        atoms_after_comp.set_pbc([True,True,True])
                                        overlapping_indices = find_overlapping_atoms( atoms_after_comp, threshold=0.3) 
                                        
                                        if overlapping_indices:
                                            # 削除関数を呼び出して initial_atoms を更新
                                            initial_atoms = remove_atoms_by_index( atoms_after_comp, overlapping_indices)
                                            print(f"  [2-1b] ✔️ 重複削除後の原子数: {len( atoms_after_comp)}") 
                                            
                                        # set_fixed_atoms_under(atoms_after_comp)
                                        
                                        # atoms_after_comp = set_interface_fixed_atoms_uniform_free(atoms_after_comp, UNIFIED_CONDITIONS["free_region_thickness"])
                                        # atoms = MatlantisAtoms.from_ase_atoms(atoms_after_comp)
                                        # atoms=atoms_after_comp.copy()
                                        # system = ASEMDSystem(atoms)
                                        # # system.init_temperature(300.0)
                                        # integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                                        # estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
                                        # md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                                        # md(system)
                                        cont_params = {
                                            'model_version': 'v7.0.0',
                                            'calc_mode': 'CRYSTAL_U0',
                                            "atoms": atoms_after_comp,
                                            "timestep": 1.0,
                                            "temperature": 300,  # 最終温度を維持
                                            "n_run": 3000,
                                            "logger_interval": 100,
                                            "traj_freq": 100,
                                            "traj_filepath":equil_file
                                        }
                            
                                        # 3. 追加のMD計算を実行
                                        run_constant_temp_md(cont_params)
                                        # 高温処理 (NVT, 10ps @ high_temp)
                                        atoms_after_equil = read(equil_file, index='-1')
                                        set_fixed_atoms_under(atoms_after_equil)
                                        atoms_after_equil = set_interface_fixed_atoms_uniform_free(atoms_after_equil, UNIFIED_CONDITIONS["free_region_thickness"])
                                        atoms = MatlantisAtoms.from_ase_atoms(atoms_after_equil)
                                        system = ASEMDSystem(atoms)
                                        system.init_temperature(float(high_temp))
                                        integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                                        estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')
                                        md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                                        md(system)
                                    else:
                                        print("高温シミュレーションは既に完了しています。")
        
                                    # ステップ3: 引張試験
                                    if not os.path.exists(summary_file):
                                        print("引張シミュレーション実行中...")
                                        final_results = run_interface_tensile_md(high_temp_file, tensile_file, analysis_file)
                                        save_interface_results(final_results, summary_file, base_name, material_info, pressure, comp_temp, high_temp)
                                        
                                        print(f"\n✅ 引張計算完了: {base_name}")
                                        print(f"📊 結果サマリー:")
                                        print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                                        print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                                    else:
                                        print("引張シミュレーションと解析は既に完了しています。")
        
                                except Exception as e:
                                    print(f"An error occurred processing {base_name} ({condition_name}): {e}")
                                    import traceback
                                    traceback.print_exc()
                                    continue
                                pass # (実際の処理)

                    # 正常に処理が完了したら、処理済みリストに追加
                    print(f"✅ 正常に処理完了: {interface_file}")
                    processed_files.add(interface_file)

                except Exception as e:
                    print(f"🚨 重大なエラー: ファイル '{interface_file}' の処理中に問題が発生しました。")
                    print(f"   エラー詳細: {e}")
                    # エラーが発生したファイルもリストに追加し、無限ループを防ぐ
                    processed_files.add(interface_file)
                
                print(f"    (処理済みカウント: {len(processed_files)}/{MAX_FILES_TO_PROCESS})")

                # 'for' ループの最後でもチェック
                if len(processed_files) >= MAX_FILES_TO_PROCESS:
                    break

            # 'while' ループの最後でもチェック (forループが正常終了した場合)
            if len(processed_files) >= MAX_FILES_TO_PROCESS:
                break # 'while' ループを抜ける

        except KeyboardInterrupt:
            print("\n[Ctrl+C] が検出されました。監視を終了します。")
            break # whileループを抜けてプログラムを終了
        except Exception as e:
            print(f"🚨予期せぬエラーが発生しました: {e}")
            print("5分後に再試行します...")
            time.sleep(300) # 5分待機

    # --- while ループが終了した後の処理 ---
    print("=" * 80)
    print(f"処理済みファイルが {len(processed_files)} 件に達しました。")
    print("NMC界面構造MD解析システムを正常に終了します。")
    print("=" * 80)


if __name__ == '__main__':
    # ... (estimator_fnなどのグローバルな設定) ...

    
    main()

✅ Matlantis (matlantis_features) モジュールを正常にインポートしました。
NMC界面構造MD解析システム (最大40件処理モード)

============================== ファイルチェック開始 (2025-10-26 05:35:46) ==============================
(現在 0 / 60 件 処理済み)
発見された界面構造ファイル数: 40
✨ 40個の新しいファイルを発見。処理を開始します。

--- 処理開始: Interface_AlF3_on_NMC_Co_decrease_001.cif ---

--- 条件: P0.000_T300K_HT500K ---
圧着シミュレーション実行中...
平衡化と高温シミュレーション(500K)を実行中...
    古い計算機 (.calc) をリセットします。
  [Check] 原子間の重なりをチェック (閾値 = 0.3 Å)...


  [Check] ✔️ 重なっている原子は見つかりませんでした。
ステップ数: 3000 (3.0 ps), 維持温度: 300.00 K


  0%|          | 0/3000 [00:00<?, ?it/s]

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/structure/MD_AlF3/heat/AlF3_NMC_Co_decrease_001_P0.000_T300K_equil.traj.


Note: The max disk size of /home/jovyan is about 98G.


...


============================== ファイルチェック開始 (2025-10-31 06:00:06) ==============================
(現在 40 / 60 件 処理済み)
発見された界面構造ファイル数: 40
新しい解析対象ファイルはありません。30秒間待機します...



============================== ファイルチェック開始 (2025-10-31 06:00:36) ==============================
(現在 40 / 60 件 処理済み)
発見された界面構造ファイル数: 40
新しい解析対象ファイルはありません。30秒間待機します...



============================== ファイルチェック開始 (2025-10-31 06:01:06) ==============================
(現在 40 / 60 件 処理済み)
発見された界面構造ファイル数: 40
新しい解析対象ファイルはありません。30秒間待機します...



============================== ファイルチェック開始 (2025-10-31 06:01:36) ==============================
(現在 40 / 60 件 処理済み)
発見された界面構造ファイル数: 40
新しい解析対象ファイルはありません。30秒間待機します...



============================== ファイルチェック開始 (2025-10-31 06:02:06) ==============================
(現在 40 / 60 件 処理済み)
発見された界面構造ファイル数: 40
新しい解析対象ファイルはありません。30秒間待機します...
